In [40]:
import pandas as pd
import string
import os
import numpy as np
from sqlalchemy import create_engine, text

from backend.utils import DbEngine, Queries
from backend.etl import ETL
from backend.etl_sdhi import ETL_SDHI
from encoding_module.encoder import Encoding

### Connect to DB

In [41]:
db_engine = DbEngine(user = 'team77', 
                     password = 'mintic2020.',
                     ip = 'postgres', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()
queries = Queries(engine)

### Loading original data

In [46]:
data = pd.read_excel('../data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)

In [ ]:
data.count()

In [ ]:
data2 = data[['INTERNOEN', 'DELITO', 'TITULO_DELITO', 'SUBTITULO_DELITO', 'TENTATIVA',
       'AGRAVADO', 'CALIFICADO', 'FECHA_INGRESO', 'FECHA_SALIDA',
       'FECHA_CAPTURA', 'SITUACION_JURIDICA', 'ANO_NACIMIENTO', 'EDAD',
       'GENERO', 'ESTADO_CIVIL', 'PAIS_INTERNO', 'DEPARTAMENTO', 'CIUDAD',
       'REINCIDENTE', 'ESTADO_INGRESO', 'ACTIVIDADES_TRABAJO',
       'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA', 'NIVEL_EDUCATIVO',
       'HIJOS_MENORES', 'CONDIC_EXPECIONAL', 'CODIGO_ESTABLECIMIENTO',
       'ESTABLECIMIENTO', 'DEPTO_ESTABLECIMIENTO', 'MPIO_ESTABLECIMIENTO',
       'REGIONAL', 'ESTADO']]
data2['AGRAVADO'].unique()

### Execute ETL

In [ ]:
etl = ETL(queries)
etl.inmate_static_info(data2)
etl.inmate_variable_info(data2)

### Execute ETL SHDI index

In [43]:
data_shdi = pd.read_excel('../data/SHDI Colombia 4.0.xlsx')

In [44]:
data_shdi.columns

Index(['iso_code', 'country', 'year', 'GDLCODE', 'level', 'region',
       'continent', 'shdi', 'healthindex', 'incindex', 'edindex', 'lifexp',
       'gnic', 'esch', 'msch', 'pop'],
      dtype='object')

In [45]:
etl_sdhi = ETL_SDHI(queries)
etl_sdhi.inmate_static_info(data_shdi)

### Encode

In [7]:
encoding = Encoding(queries)
data_2_encode = encoding.get_data('encoding')
data_encoded = encoding.one_hot(data_2_encode)
data_encoded

,internoen,genero,nacionalidad,estado_civil,nivel_educativo,reconocimiento_etnico,diversidad_sexual,extranjero,estado_ingreso,fecha_ingreso,...,delito_LESIONES PERSONALES,delito_OTRO,delito_TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,delito_VIOLENCIA INTRAFAMILIAR,tentativa_1,tentativa_2,agravado_1,agravado_2,calificado_1,calificado_2
0,2660D0E0F43523200EF105604266F18D6CA396E5,2,22,3,3,5,1,1,4,2010-01-03,...,0,0,1,0,1,0,1,0,1,0
1,82C211443D1DD143DE694380BDE42BA3F210101A,1,22,3,5,5,1,1,4,2010-01-06,...,0,0,0,0,1,0,0,1,0,1
2,F3C098B7A4D023E829D971232432B68A0299D895,2,22,3,2,5,1,1,4,2010-01-07,...,0,0,0,0,1,0,0,1,0,1
3,07CA56D66F9EBB7330BB9FD6AD2FB405E246033B,2,22,3,3,5,1,1,5,2010-01-12,...,0,0,1,0,1,0,1,0,1,0
4,72352CBB570F2D72DF176A2D469B1C838F58A088,2,22,3,3,5,1,1,4,2010-01-26,...,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136453,43B91358C7A906DA26DBEC3426069D04449FE755,2,22,1,2,5,1,1,4,2020-04-29,...,0,1,0,0,1,0,1,0,1,0
136454,67BC95487C433D55DAA1D9D82FE1B1821D2EAB82,2,22,6,4,5,1,1,4,2020-05-22,...,0,1,0,0,1,0,1,0,1,0
136455,A14C27EF95AB3A18C5CE2A2319E589003C5329B3,2,22,7,3,5,1,1,4,2020-05-23,...,0,0,0,0,1,0,1,0,1,0
136456,66B99073712FDB3287F1925FED25A30D43736A69,2,22,7,5,5,1,1,4,2020-06-03,...,0,0,0,0,1,0,0,1,0,1
